data download(http://yongyeol.com/2011/12/15/paper-flavor-network.html)

In [2]:
import org.apache.spark.graphx._
import org.apache.spark.rdd._

In [3]:
import scala.io.Source

In [4]:
Source.fromFile("./data/ingr_info.tsv").getLines().take(7).foreach(println)

# id	ingredient name	category
0	magnolia_tripetala	flower
1	calyptranthes_parriculata	plant
2	chamaecyparis_pisifera_oil	plant derivative
3	mackerel	fish/seafood
4	mimusops_elengi_flower	flower
5	hyssop	herb


In [5]:
Source.fromFile("./data/comp_info.tsv").getLines().take(7).foreach(println)

# id	Compound name	CAS number
0	jasmone	488-10-8
1	5-methylhexanoic_acid	628-46-6
2	l-glutamine	56-85-9
3	1-methyl-3-methoxy-4-isopropylbenzene	1076-56-8
4	methyl-3-phenylpropionate	103-25-3
5	3-mercapto-2-methylpentan-1-ol_(racemic)	227456-27-1


In [6]:
Source.fromFile("./data/ingr_comp.tsv").getLines().take(7).foreach(println)

# ingredient id	compound id
1392	906
1259	861
1079	673
22	906
103	906
1005	906


In [7]:
class FNNode(val name: String)
case class Ingredient(override val name: String, category: String) extends FNNode(name)
case class Compound(override val name: String, cas: String) extends FNNode(name)

In [8]:
val ingredients: RDD[(VertexId, FNNode)] = 
sc.textFile("./data/ingr_info.tsv").
      filter(! _.startsWith("#")).
      map {line => 
             val row = line split '\t'
             (row(0).toInt, Ingredient(row(1), row(2)))
          } 

In [9]:
val compounds: RDD[(VertexId, FNNode)] = 
sc.textFile("./data/comp_info.tsv").
      filter(! _.startsWith("#")).
      map {line => 
             val row = line split '\t'
             (10000L + row(0).toInt, Compound(row(1), row(2)))
          }     

In [10]:
val links: RDD[Edge[Int]] = 
  sc.textFile("./data/ingr_comp.tsv").
     filter(! _.startsWith("#")).
     map {line => 
        val row = line split '\t'
        (Edge(row(0).toInt, 10000L + row(1).toInt, 1))
     }

In [15]:
val nodes = ingredients++compounds
val foodNetwork = Graph(nodes, links)

In [16]:
def showTriplet(t: EdgeTriplet[FNNode,Int]): String =
"The ingredient " ++ t.srcAttr.name ++ " contains " ++ t.dstAttr.name

In [ ]:
foodNetwork.triplets.take(1).
     foreach(showTriplet _ andThen println _)